In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Caching - Small file').getOrCreate()

25/06/12 03:58:47 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
!hdfs dfs -ls /data/

Found 7 items
-rw-r--r--   2 root hadoop      12174 2025-06-09 03:26 /data/customers1.csv
-rw-r--r--   2 root hadoop    1048576 2025-06-03 03:00 /data/customers10.csv
-rw-r--r--   2 root hadoop    1048576 2025-06-05 03:18 /data/customers150.csv
-rw-r--r--   2 root hadoop    1048576 2025-06-04 03:32 /data/customers500.csv
drwxr-xr-x   - root hadoop          0 2025-06-10 05:01 /data/external_data
-rw-r--r--   2 root hadoop        511 2025-06-06 02:33 /data/malformed_customers.csv
drwxr-xr-x   - root hadoop          0 2025-06-07 08:08 /data/write_op


In [8]:
rdd = spark.sparkContext.textFile('/data/customers1.csv')

In [9]:
rdd.first()

'India,2023-10-07,False'

In [13]:
header = rdd.first()
rdd_no_header = rdd.filter(lambda row : (row != header)).map(lambda row : row.split(','))

In [16]:
reducedCity = rdd_no_header.map(lambda row : (row[2], 1)).reduceByKey(lambda x, y : x+y)

In [17]:
reducedCity.collect()

[('Delhi', 25),
 ('Hyderabad', 29),
 ('Pune', 17),
 ('Mumbai', 30),
 ('Bangalore', 19),
 ('Kolkata', 28),
 ('Chennai', 27),
 ('Ahmedabad', 23)]

In [18]:
reducedCity.collect()

[('Delhi', 25),
 ('Hyderabad', 29),
 ('Pune', 17),
 ('Mumbai', 30),
 ('Bangalore', 19),
 ('Kolkata', 28),
 ('Chennai', 27),
 ('Ahmedabad', 23)]

### cache()

In [21]:
reducedCity.cache() # by default cache is LAZY

PythonRDD[21] at collect at /tmp/ipykernel_18924/979138697.py:1

In [22]:
reducedCity.count()

8

In [23]:
# Now you can see the cache in YARN ResourceManager -> Storage

In [ ]:
reducedCity.unpersist() #to uncache